<a href="https://colab.research.google.com/github/Shreyabaral/SummarizeTube/blob/main/SummarizeTube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import streamlit as st
from langchain.document_loaders import YoutubeLoader

from youtube_transcript_api import YouTubeTranscriptApi
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
from langchain.chains.summarize import load_summarize_chain
from langchain.llms import HuggingFacePipeline

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

import torch

import langchain
print(langchain.__version__)

In [ ]:
#get the transcript from the video url
def get_transcript(youtube_url):
    try:
        loader = YoutubeLoader.from_youtube_url(youtube_url)
        transcript = loader.load()
    except:
            # If no transcript is found, raise an exceptionto-
            raise Exception("No suitable transcript found.")
    return transcript

In [ ]:
transcript= get_transcript("https://www.youtube.com/watch?v=-CmadmM5cOk")

In [ ]:
new_list= str(transcript).split('.')
print(new_list)
type(new_list)

In [ ]:
!pip install huggingface_hub

In [ ]:
# using the huggingface google/flan-t5-base to see its performance

from langchain import HuggingFaceHub, LLMChain
from langchain import PromptTemplate
from transformers import BartForConditionalGeneration, BartTokenizer


template = """Summarize: {question}

Answer: """
prompt = PromptTemplate(
    template=template,

    input_variables= [question]
)

# user question
question = input("How can I help you today? : ")


# initialize Hub LLM
hub_llm = HuggingFaceHub(
        repo_id='google/flan-t5-base',
    model_kwargs={'temperature':1}
)

# create prompt template > LLM chain
llm_chain = LLMChain(
    prompt=prompt,
    llm=hub_llm
)

# ask the user question about NFL 2010
print(llm_chain.run(question))

In [ ]:
#using transformers for summarisation task


from transformers import BartForConditionalGeneration, BartTokenizer

# Define the template
template = """Summarize: {question}

Answer: {answer}"""

# Define the question
question = input("How can I help you today?")

# Load the model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Tokenize the input question
input_ids = tokenizer.encode(question, return_tensors="pt", max_length=1024, truncation=True)

# Generate the summary
summary_ids = model.generate(input_ids, max_length=150, num_beams=4, length_penalty=2.0, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Substitute the question and answer in the template
filled_template = template.format(question=question, answer=summary)

# Print the result
print(summary)

In [ ]:
#Using Stremlit for UI

In [ ]:
%%writefile app.py

#import necessary libraries
import streamlit as st
from transformers import pipeline
from transformers import BartForConditionalGeneration, BartTokenizer

#define main function for streamlit
def main():
    st.title("Your video summariser").  #set title for streamlit

    # Create a text input for user messages
    question = st.text_input("You:", "").  #get the youtube video URL

    if question:                           #checks if user has entered the question
        # Generate a response
        response = generate_response(question).        # generates the response base on defined function

        # Display the bot's response
        st.text_area("Bot:", response, height=100).     #Displays the response in text area


#fucntion to genrate the response using  Language Model
def generate_response(question):

    # Define the template for formatting response
    template = """summarize_with_langchain_and_openai {question}

    Answer: {answer}"""


    # set the input
    question = question

    # Load the pre-trained model and tokenizer
    model_name = "facebook/bart-large-cnn"
    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)

    # Tokenize the input question
    input_ids = tokenizer.encode(question, return_tensors="pt", max_length=1500, truncation=True)

    # Generate the summary using the model
    summary_ids = model.generate(input_ids, max_length=500, num_beams=4, length_penalty=1.0, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

if __name__ == "__main__":
    main()



In [ ]:
## Install the 'localtunnel' package using npm
!npm install localtunnel


In [ ]:
# Run the Streamlit app in the background, redirecting both standard output and error to a log file
##Use curl to create a public tunnel using localtunnel, accessing the generated tunnel with a specific password

!streamlit run /content/app.py &>/content/logs.txt & curl https://loca.lt/mytunnelpassword

In [ ]:
# Use npx to run localtunnel, creating a public tunnel for the specified port (8501 by default)
!npx localtunnel --port 8501

^C
